In [1]:
import pulp as pl
import numpy as np
from pathlib import Path
from itertools import product

In [2]:
resource_names = ["ore", "clay", "obsidian", "geode"]

In [3]:
def solve_robot_ilp(
    name: str,
    ore_cost: int,
    clay_cost: int,
    obsidian_costs: tuple[int, int],
    geode_costs: tuple[int, int],
    num_timesteps: int,
):
    print(
        f"{name} for {num_timesteps} timesteps: {ore_cost}, {clay_cost}, {obsidian_costs}, {geode_costs}"
    )
    ilp = pl.LpProblem(name, sense=pl.LpMaximize)

    new_robots = pl.LpVariable.dicts(
        name="new_robots",
        indices=(resource_names, range(num_timesteps + 1)),
        lowBound=0,
        cat=pl.LpInteger,
    )
    resource_gain = pl.LpVariable.dicts(
        name="gained",
        indices=(resource_names, range(num_timesteps + 1)),
        lowBound=0,
        cat=pl.LpInteger,
    )
    resource_spent = pl.LpVariable.dicts(
        name="spent",
        indices=(resource_names, range(num_timesteps + 1)),
        lowBound=0,
        cat=pl.LpInteger,
    )

    def _set_var(var: pl.LpVariable, val: int):
        var.setInitialValue(val)
        var.fixValue()

    for res_name in resource_names:
        _set_var(new_robots[res_name][0], 1 if res_name == "ore" else 0)
        _set_var(resource_gain[res_name][0], 0)
        _set_var(resource_spent[res_name][0], 0)

    ilp += (
        pl.lpSum(resource_gain["geode"]),
        "cost_func",
    )

    for time in range(1, num_timesteps + 1):
        for res_name in resource_names:
            # sum of all gains must be larger or equal to sum of all spent resources
            ilp += (
                pl.lpSum([resource_gain[res_name][tt] for tt in range(time + 1)])
                >= pl.lpSum([resource_spent[res_name][tt] for tt in range(time + 1)]),
                f"no_debt_{res_name}_{time}",
            )

        for res_name in resource_names:
            ilp += (
                resource_gain[res_name][time]
                == pl.lpSum(new_robots[res_name][tt] for tt in range(time)),
                f"{res_name}_gain_{time}",
            )

        ilp += (
            pl.lpSum(new_robots[res_name][time] for res_name in resource_names) <= 1,
            f"build_only_one_robot_{time}",
        )

        ilp += (
            pl.lpDot(
                [new_robots[res_name][time] for res_name in resource_names],
                [ore_cost, clay_cost, obsidian_costs[0], geode_costs[0]],
            )
            == resource_spent["ore"][time - 1],
            f"ore_spent_{time}",
        )

        ilp += (
            obsidian_costs[1] * new_robots["obsidian"][time]
            == resource_spent["clay"][time - 1],
            f"clay_spent_{time}",
        )
        ilp += (
            geode_costs[1] * new_robots["geode"][time]
            == resource_spent["obsidian"][time - 1],
            f"obsidian_spent_{time}",
        )

    ilp.writeLP(f"aoc22_day19_{name}.lp")

    res = ilp.solve(pl.COIN_CMD(msg=False))
    if res == pl.LpStatusInfeasible:
        print("INFEASIBLE")
    elif res == pl.LpSolutionUnbounded:
        print("UNBOUNDED")

    return resource_gain, resource_spent, new_robots, ilp

In [4]:
def print_ilp(name, num_timesteps, new_robots, resource_gain, resource_spent):
    print(name)
    for time in range(num_timesteps + 1):
        print(f"== Minute {time} ==")
        for res in resource_names:
            new_robs = sum(pl.value(new_robots[res][tt]) for tt in range(time + 1))
            if new_robs > 0:
                res_budget = sum(
                    pl.value(resource_gain[res][tt]) - pl.value(resource_spent[res][tt])
                    for tt in range(time + 1)
                )
                print(
                    f"{new_robs} {res}-collecting robot collects {pl.value(resource_gain[res][time])} {res}; "
                    f"you now have {res_budget} {res}"
                )
        print()

In [5]:
def parse_input(
    filename: str, num_timesteps: int = 24, max_blueprints: int | None = None
):
    val_lst = []
    with open(filename) as f:
        for row_idx, row in enumerate(f):
            if max_blueprints and row_idx >= max_blueprints:
                break
            row = row.rstrip().split(" ")
            ore_cost = int(row[6])
            clay_cost = int(row[12])
            obsidian_costs = (int(row[18]), int(row[21]))
            geode_costs = (int(row[27]), int(row[30]))
            name = f"{Path(filename).stem}_{row_idx}"
            resource_gain, resource_spent, new_robots, ilp = solve_robot_ilp(
                name, ore_cost, clay_cost, obsidian_costs, geode_costs, num_timesteps
            )
            # print_ilp(name, num_timesteps, new_robots, resource_gain, resource_spent)
            val_lst.append(ilp.objective.value())
    return val_lst, np.dot(val_lst, np.arange(1, len(val_lst) + 1))

In [6]:
parse_input("test-input.txt", 24)

test-input_0 for 24 timesteps: 4, 2, (3, 14), (2, 7)
test-input_1 for 24 timesteps: 2, 3, (3, 8), (3, 12)


([9.0, 12.0], 33.0)

In [7]:
parse_input("input.txt", 24)[-1]

input_0 for 24 timesteps: 4, 3, (3, 18), (4, 8)
input_1 for 24 timesteps: 4, 4, (2, 11), (2, 7)
input_2 for 24 timesteps: 4, 4, (4, 5), (2, 10)
input_3 for 24 timesteps: 3, 3, (3, 8), (2, 12)
input_4 for 24 timesteps: 4, 4, (4, 17), (4, 16)
input_5 for 24 timesteps: 2, 4, (4, 20), (4, 18)
input_6 for 24 timesteps: 3, 4, (2, 20), (4, 7)
input_7 for 24 timesteps: 3, 4, (3, 6), (2, 10)
input_8 for 24 timesteps: 4, 3, (3, 14), (4, 17)
input_9 for 24 timesteps: 2, 4, (3, 19), (4, 8)
input_10 for 24 timesteps: 4, 3, (2, 13), (2, 10)
input_11 for 24 timesteps: 2, 2, (2, 20), (2, 14)
input_12 for 24 timesteps: 2, 4, (4, 18), (2, 11)
input_13 for 24 timesteps: 4, 3, (3, 11), (4, 7)
input_14 for 24 timesteps: 2, 4, (4, 17), (3, 11)
input_15 for 24 timesteps: 4, 3, (3, 20), (2, 19)
input_16 for 24 timesteps: 3, 4, (3, 18), (4, 19)
input_17 for 24 timesteps: 2, 2, (2, 10), (2, 11)
input_18 for 24 timesteps: 4, 4, (2, 18), (4, 20)
input_19 for 24 timesteps: 4, 4, (2, 9), (3, 15)
input_20 for 24 tim

1613.0

In [8]:
parse_input("test-input.txt", 32, max_blueprints=3)

test-input_0 for 32 timesteps: 4, 2, (3, 14), (2, 7)
test-input_1 for 32 timesteps: 2, 3, (3, 8), (3, 12)


([56.0, 62.0], 180.0)

In [9]:
val_lst, quality_score = parse_input("input.txt", 32, max_blueprints=3)
print(val_lst, np.prod(val_lst))

input_0 for 32 timesteps: 4, 3, (3, 18), (4, 8)
input_1 for 32 timesteps: 4, 4, (2, 11), (2, 7)
input_2 for 32 timesteps: 4, 4, (4, 5), (2, 10)
[28.0, 38.0, 44.0] 46816.0
